In [46]:
import os
import json
import datetime
import time
import re
from pathlib import Path

import google.generativeai as genai
import pdfplumber
import pymupdf4llm

load_dotenv()

class PaperAnalyzer:
    def __init__(self):

        api_key = os.getenv("GEMINI_AI_API_KEY")
        if not api_key:
            raise ValueError("API key not found. Please set it in the .env file.")
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('gemini-1.5-pro')
        self.paper_text = None
        self.execution_times = {
            "claims_analysis": 0,
            "evidence_analysis": 0,
            "conclusions_analysis": 0,
            "total_time": 0,
            "total_sleep_time": 0,
            "actual_processing_time": 0
        }

    def extract_text_from_pdf(self, filename: str) -> str:
        """Extract text from PDF file using PyMuPDF"""
        try:
            self.paper_text = pymupdf4llm.to_markdown(filename)
            return self.paper_text
        except Exception as e:
            print(f"Error extracting text from PDF: {e}")
            return ""

    def get_claims(self, filename: str) -> dict:
        """Extract all claims from the paper"""
        if not self.paper_text:
            text = self.extract_text_from_pdf(filename)
        else:
            text = self.paper_text
        start_time = time.time()
        if not self.paper_text:
            raise Exception("Failed to extract text from PDF")
        
        claims_prompt = f"""
        Analyze this research paper and extract ALL possible claims made by the authors.
        Paper text: {text}
        
        Your task is to identify all statements in the text that meet the following criteria for a claim:
        1. Makes a specific, testable assertion about results, methods, or contributions
        2. Represents a novel finding, improvement, or advancement
        3. Presents a clear position or conclusion
        
        Make sure to:
        1. Include both major and minor claims
        2. Don't miss any claims
        3. Present each claim as a separate item
        4. Don't repear any claim
        
        Return ONLY the following JSON structure:
        {{
            "claims": [
                {{
                    "claim_id": 1,
                    "claim_text": "statement of the claim",
                    "location": "section/paragraph where this claim appears",
                    "claim_type": "Nature of the claim",
                    "exact_quote": "complete verbatim text containing the claim"
                }}
            ]
        }}
        """
        time.sleep(120)
        self.execution_times["total_sleep_time"] += 120
        response = self.model.generate_content(claims_prompt)
        self.execution_times["claims_analysis"] = time.time() - start_time
        print(response.text)
        return self._parse_json_response(response.text)

    def analyze_evidence(self, filename: str, claims: dict) -> list:
        """Find evidence for each claim"""
        if not self.paper_text:
            text = self.extract_text_from_pdf(filename)
        else:
            text = self.paper_text
        start_time = time.time()
        evidence_results = []
        for claim in claims['claims']:
            evidence_prompt = f"""
            Paper text: {text}
            
            For the following claim from the paper:
            "{claim['claim_text']}"
            
            Please identify relevant evidence that:
            1. Directly supports or contradicts the claim's specific assertion
            2. Is presented with experimental results, data, or concrete examples
            3. Can be traced to specific methods, results, or discussion sections
            4. Is not from the abstract or introduction
            
            If NO evidence is found for the given Claim, return:
            {{
                "claim_id": {claim['claim_id']},
                "evidence": [],
                "no_evidence_reason": "Explain why no evidence was found (e.g., 'Claim is unsupported', 'Claim is theoretical without empirical evidence', etc.)"
            }}
            ELSE:
            Return ONLY the following JSON structure:
            {{
                "claim_id": {claim['claim_id']},
                "evidence": [
                    {{
                        "evidence_id": 1,
                        "evidence_text": "specific experimental result/data point",
                        "evidence_type": "primary/secondary",
                        "strength": "strong/moderate/weak",
                        "limitations": "stated limitations or assumptions",
                        "location": "specific section & paragraph",
                        "exact_quote": "verbatim text from paper"
                    }}
                ]
            }}
            """
            time.sleep(120)
            self.execution_times["total_sleep_time"] += 120
            response = self.model.generate_content(evidence_prompt)
            result = self._parse_json_response(response.text)
            if result:
                evidence_results.append(result)
        self.execution_times["evidence_analysis"] = time.time() - start_time
        print(evidence_results)
        return evidence_results

    def analyze_conclusions(self, filename: str, claims: dict, evidence_results: list) -> dict:
        """Analyze conclusions considering claims and evidence one at a time"""
        if not self.paper_text:
            text = self.extract_text_from_pdf(filename)
        else:
            text = self.paper_text
        start_time = time.time()
        all_conclusions = []
        claims_list = claims.get('claims', [])
        for claim in claims_list:
            claim_id = claim.get('claim_id')
            claim_evidence = next((e['evidence'] for e in evidence_results if e.get('claim_id') == claim_id), [])
            evidence_text = []
            for idx, evidence in enumerate(claim_evidence, 1):
                evidence_text.append(
                    f"  Evidence {idx}:\n"
                    f"    - Text: {evidence.get('evidence_text', 'No text provided')}\n"
                    f"    - Strength: {evidence.get('strength', 'Not specified')}\n"
                    f"    - Limitations: {evidence.get('limitations', 'None specified')}\n"
                    f"    - Location: {evidence.get('location', 'Location not specified')}"
                )
            single_analysis = f"""
            Claim {claim_id}:
            Statement: {claim.get('claim_text', 'No text provided')}
            Location: {claim.get('location', 'Location not specified')}
            
            Evidence Summary:
            {{'\n'.join(evidence_text)}}
            """
            single_conclusion_prompt = f"""
            Paper text: {text}
            
            Analyze the following claim and its supporting evidence:
            {single_analysis}
            
            Provide a comprehensive conclusion analysis following these guidelines:
            
            1. Evidence Assessment:
            - Evaluate the strength and quality of ALL evidence presented
            - Consider both supporting and contradicting evidence
            - Assess the methodology and reliability of evidence
            
            2. Conclusion Analysis:
            - Determine what the authors concluded about this specific claim
            - Evaluate if the conclusion is justified by the evidence
            - Consider the relationship between evidence quality and conclusion strength
            
            3. Robustness Evaluation:
            - Assess how well the evidence supports the conclusion
            - Consider methodological strengths and weaknesses
            - Evaluate the consistency of evidence
            
            4. Limitations Analysis:
            - Identify specific limitations in both evidence and conclusion
            - Consider gaps in methodology or data
            - Note any potential biases or confounding factors
            
            Return ONLY the following JSON structure:
            {{
                "conclusions": [
                    {{
                        "claim_id": "{claim_id}",
                        "author_conclusion": "detailed description of authors' conclusion based on evidence",
                        "conclusion_justified": true/false,
                        "justification_explanation": "detailed explanation of why conclusion is/isn't justified",
                        "robustness_analysis": "comprehensive analysis of evidence strength and reliability",
                        "limitations": "specific limitations and caveats",
                        "location": "section/paragraph where conclusion appears",
                        "evidence_alignment": "analysis of how well evidence aligns with conclusion",
                        "confidence_level": "high/medium/low based on evidence quality"
                    }}
                ]
            }}
            """
            try:
                time.sleep(120)
                self.execution_times["total_sleep_time"] += 120
                response = self.model.generate_content(single_conclusion_prompt)
                result = self._parse_json_response(response.text)
                if result and isinstance(result, dict) and 'conclusions' in result and result['conclusions']:
                    conclusion = result['conclusions'][0]
                    all_conclusions.append(conclusion)
                else:
                    all_conclusions.append({
                        "claim_id": claim_id,
                        "author_conclusion": "No conclusion available",
                        "conclusion_justified": False,
                        "justification_explanation": "Analysis not available",
                        "robustness_analysis": "No robustness analysis available",
                        "limitations": "No limitations analysis available",
                        "location": "Location not specified",
                        "evidence_alignment": "No alignment analysis available",
                        "confidence_level": "low"
                    })
            except Exception as e:
                print(f"Error analyzing conclusion for claim {claim_id}: {str(e)}")
                all_conclusions.append({
                    "claim_id": claim_id,
                    "author_conclusion": "Error in analysis",
                    "conclusion_justified": False,
                    "justification_explanation": "Analysis failed",
                    "robustness_analysis": "Analysis failed",
                    "limitations": "Analysis failed",
                    "location": "Location not specified",
                    "evidence_alignment": "Analysis failed",
                    "confidence_level": "low"
                })
        self.execution_times["conclusions_analysis"] = time.time() - start_time
        return {
            "conclusions": all_conclusions,
            "analysis_metadata": {
                "total_claims_analyzed": len(claims_list),
                "claims_with_conclusions": len(all_conclusions),
                "analysis_timestamp": str(datetime.datetime.now())
            }
        }
        
    def _parse_json_response(self, response: str) -> dict:
        try:
            response = response.strip()
            response = re.sub(r'^`*``json|```*$', '', response).strip()
            if not response.startswith('{'):
                response = '{' + response
            if not response.endswith('}'):
                response += '}'
            response = re.sub(r"([{\[,:\s])(['])((?:[^'\\\\]|\\\\.)*)(\2)([}\],\s])", r'\1"\3"\5', response)
            response = re.sub(r'\\(?!["\\bfnrt/])', r'\\\\', response)
            response = re.sub(r'([{,]\s*)(\w+)(\s*:)', lambda m: f'{m.group(1)}"{m.group(2)}"{m.group(3)}', response)
            response = re.sub(r',\s*([}\]])', r'\1', response)
            response = re.sub(r'\bNaN\b', 'null', response)
            response = re.sub(r'\b(?:Infinity|-Infinity)\b', 'null', response)
            result = json.loads(response)
            return result if isinstance(result, dict) else {}
        except json.JSONDecodeError as e:
            error_position = e.pos
            snippet_start = max(error_position - 20, 0)
            snippet_end = min(error_position + 20, len(response))
            error_snippet = response[snippet_start:snippet_end]
            print(f"JSON parsing failed at position {error_position}: {e.msg}")
            print(f"Error snippet: ...{error_snippet}...")
            return {'error': 'Failed to parse JSON', 'reason': str(e)}
        except Exception as e:
            print(f"Unexpected error during JSON parsing: {str(e)}")
            return {'error': 'Unexpected parsing error', 'exception': str(e)}
            
    def combine_results(self, claims: dict, evidence_results: list, conclusions: dict) -> dict:
        final_results = {
            "paper_analysis": []
        }
        conclusions_dict = { c['claim_id']: c for c in conclusions.get('conclusions', []) } if conclusions else {}
        evidence_dict = { e['claim_id']: e.get('evidence', []) for e in evidence_results if isinstance(e, dict) }
        for claim in claims.get('claims', []):
            claim_id = claim['claim_id']
            conclusion = conclusions_dict.get(claim_id, {})
            evidence = evidence_dict.get(claim_id, [])
            analysis = {
                "claim_id": claim_id,
                "claim": claim.get('claim_text', ''),
                "claim_location": claim.get('location', 'Location not specified'),
                "evidence": evidence,
                "evidence_locations": [ev.get('location', 'Location not specified') for ev in evidence],
                "conclusion": {
                    "author_conclusion": conclusion.get('author_conclusion', 'No conclusion available'),
                    "conclusion_justified": conclusion.get('conclusion_justified', False),
                    "robustness_analysis": conclusion.get('robustness_analysis', 'No robustness analysis available'),
                    "limitations": conclusion.get('limitations', 'No limitations analysis available'),
                    "conclusion_location": conclusion.get('location', 'Location not specified')
                }
            }
            final_results['paper_analysis'].append(analysis)
        final_results["execution_times"] = {
            "claims_analysis_time": f"{self.execution_times['claims_analysis']:.2f} seconds",
            "evidence_analysis_time": f"{self.execution_times['evidence_analysis']:.2f} seconds",
            "conclusions_analysis_time": f"{self.execution_times['conclusions_analysis']:.2f} seconds",
            "total_sleep_time": f"{self.execution_times['total_sleep_time']:.2f} seconds",
            "actual_processing_time": f"{(self.execution_times['total_time'] - self.execution_times['total_sleep_time']):.2f} seconds",
            "total_execution_time": f"{self.execution_times['total_time']:.2f} seconds"
        }
        return final_results
        
    def print_analysis_results(self, final_results: dict):
        print("\n=== Complete Paper Analysis ===\n")
        for analysis in final_results['paper_analysis']:
            print(f"Claim {analysis['claim_id']}:")
            print(f"Statement: {analysis['claim']}")
            print("\nEvidence:")
            for evidence in analysis['evidence']:
                print(f"- {evidence['evidence_text']}")
                print(f"  Strength: {evidence['strength']}")
                print(f"  Limitations: {evidence['limitations']}")
            print("\nConclusion:")
            print(f"Author's Conclusion: {analysis['conclusion']['author_conclusion']}")
            print(f"Justified by Evidence: {'Yes' if analysis['conclusion']['conclusion_justified'] else 'No'}")
            print(f"Robustness: {analysis['conclusion']['robustness_analysis']}")
            print(f"Limitations: {analysis['conclusion']['limitations']}")
            print("\n" + "-"*50 + "\n")


def results_exist(base_filename: str, output_folder: str) -> bool:
    """Check if results already exist for the given file."""
    output_dir = Path(output_folder)
    analysis_path = output_dir / f'{base_filename}_analysis.json'
    # Check if all expected output files exist
    return any(file.exists() for file in [analysis_path])


def main():
    import datetime
    analyzer = PaperAnalyzer()
    input_folder = 'shashi_1_papers_trimmed'
    output_folder = 'Gemini_one_by_one_shashi'
    pdf_files = [f for f in os.listdir(input_folder) if f.endswith('.pdf')]
    
    for filename in pdf_files:
        basefile_name = Path(filename).stem

        
        if results_exist(basefile_name, output_folder):
            print(f"Skipping {filename}, results already exist.")
            continue
        
        try:
            filename = f"{input_folder}/{filename}"
            final_output_path = f'Gemini_one_by_one_shashi/{basefile_name}_analysis.json'
            intermediate_output_path = f'Gemini_one_by_one_shashi/{basefile_name}_intermediate.json'
            if os.path.exists(final_output_path):
                print(f"Skipping {filename}, already processed.")
                continue
            total_start_time = time.time()
            print("Extracting claims...")
            claims = analyzer.get_claims(filename)
            print("Analyzing evidence...")
            evidence_results = analyzer.analyze_evidence(filename, claims)
            print("Analyzing conclusions...")
            conclusions = analyzer.analyze_conclusions(filename, claims, evidence_results)
            analyzer.execution_times["total_time"] = time.time() - total_start_time
            final_results = analyzer.combine_results(claims, evidence_results, conclusions)
            analyzer.print_analysis_results(final_results)
            os.makedirs('Gemini_one_by_one_shashi', exist_ok=True)
            with open(f'Gemini_one_by_one_shashi/{basefile_name}_analysis.json', 'w') as f:
                json.dump(final_results, f, indent=4)
            os.makedirs('Gemini_one_by_one_shashi', exist_ok=True)
            intermediate_results = {
                "claims": claims,
                "evidence": evidence_results,
                "conclusions": conclusions,
                "execution_times": final_results["execution_times"]
            }
            with open(f'Gemini_one_by_one_shashi/{basefile_name}_intermediate.json', 'w') as f:
                json.dump(intermediate_results, f, indent=4)
            print("Intermediate results saved to 'Gemini_one_by_one' folder")
        except Exception as e:
            print(f"Error analyzing paper: {str(e)}")

if __name__ == "__main__":
    main()


Skipping 2502.12568v2.pdf, results already exist.
Extracting claims...
Processing shashi_1_papers_trimmed/2409.15915v1.pdf...
[                                        ] (0/1=[==                                      ] ( 1/19=[====                                    ] ( 2/19=[======                                  ] ( 3/19=[========                                ] ( 4/19=[==========                              ] ( 5/19=[============                            ] ( 6/19=[==============                          ] ( 7/19=[================                        ] ( 8/19=[==================                      ] ( 9/19==[=====================                   ] (10/1=[=======================                 ] (11/1=[=========================               ] (12/1=[===========================             ] (13/1=[=============================           ] (14/1=[===============================         ] (15/1=[=================================       ] (16/1=[===============================

KeyboardInterrupt: 

In [43]:
import google.generativeai as genai
import json
from pathlib import Path
import pymupdf4llm
import time
import datetime
import logging
from typing import Dict, List, Any
import re
import os, time

load_dotenv()

class SinglePassPaperAnalyzer:
    def __init__(self):

        api_key = os.getenv("GEMINI_AI_API_KEY")
        if not api_key:
            raise ValueError("API key not found. Please set it in the .env file.")
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('gemini-1.5-pro')
        self.paper_text = None
        self.execution_times = {
        "single_pass_analysis": 0,
        "total_time": 0,
        "total_sleep_time": 0,  # Add this
        "actual_processing_time": 0  # Add this
    }
        
    def extract_text_from_pdf(self, filename: str) -> str:
        """Extract text from PDF file using PyMuPDF"""
        try:
            self.paper_text = pymupdf4llm.to_markdown(filename)
            return self.paper_text
        except Exception as e:
            print(f"Error extracting text from PDF: {e}")
            return ""

    def analyze_paper(self, filename):
        """Perform comprehensive single-pass analysis of the paper with improved error handling"""
        start_time = time.time()

        if not self.paper_text:
            text = self.extract_text_from_pdf(filename)
        else:
            text = self.paper_text
                
        if not text:
            raise Exception("Failed to extract text from PDF")
        
        # Split text into chunks if it's too long
        max_chunk_length = 12000  # Adjust based on model's context window
        chunks = [text[i:i + max_chunk_length] for i in range(0, len(text), max_chunk_length)]
        
        all_analyses = []
        claim_id_counter = 1
        
        for chunk_idx, chunk in enumerate(chunks):
            # Create chunk indicator text
            chunk_indicator = "This is part " + str(chunk_idx + 1) + " of " + str(len(chunks)) + "." if len(chunks) > 1 else ""
            portion_text = "portion of the" if len(chunks) > 1 else ""
            
            comprehensive_prompt = f"""
            Analyze this {portion_text} research paper and provide a comprehensive evaluation.
            {chunk_indicator}
            
            Paper text: {chunk}

            Follow these guidelines:

            1. Identify ALL claims in this text where each claim:
            - Makes a specific, verifiable assertion
            - Is supported by concrete evidence
            - Represents findings, contributions, or methodological advantages
            - Can be from any section except abstract

            2. For each identified claim:
            - Extract ALL supporting or contradicting evidence
            - Evaluate the evidence strength and limitations
            - Assess how well conclusions align with evidence

            Return ONLY a valid JSON object with the following structure:
            {{
                "analysis": [
                    {{
                        "claim_id": number,
                        "claim": {{
                            "text": "statement of the claim",
                            "type": "methodology/result/contribution/performance",
                            "location": "section/paragraph",
                            "exact_quote": "verbatim text from paper"
                        }},
                        "evidence": [
                            {{
                                "evidence_text": "specific experimental result/data",
                                "strength": "strong/moderate/weak",
                                "limitations": "specific limitations",
                                "location": "section/paragraph",
                                "exact_quote": "verbatim text from paper"
                            }}
                        ],
                        "evaluation": {{
                            "conclusion_justified": true/false,
                            "robustness": "high/medium/low",
                            "justification": "explanation",
                            "key_limitations": "critical limitations",
                            "confidence_level": "high/medium/low"
                        }}
                    }}
                ]
            }}

            Ensure the response is a properly formatted JSON object.
            """
            
            # Add rate limiting
            time.sleep(90)
            self.execution_times["total_sleep_time"] += 90
            
            try:
                # Get response from Gemini
                response = self.model.generate_content(comprehensive_prompt)
                
                # Parse response
                chunk_results = self._parse_json_response(response.text)
                
                if chunk_results and 'analysis' in chunk_results:
                    # Update claim IDs to ensure uniqueness across chunks
                    for analysis in chunk_results['analysis']:
                        analysis['claim_id'] = claim_id_counter
                        claim_id_counter += 1
                    all_analyses.extend(chunk_results['analysis'])
            
            except Exception as e:
                logging.error(f"Error processing chunk {chunk_idx + 1}: {str(e)}")
                continue

        if not all_analyses:
            raise Exception("No valid analyses generated from any chunk")

        result = {"analysis": all_analyses}
        
        elapsed_time = time.time() - start_time
        self.execution_times["single_pass_analysis"] = elapsed_time
        self.execution_times["actual_processing_time"] = elapsed_time - self.execution_times["total_sleep_time"]

        return result

    def _parse_json_response(self, response: str) -> Dict:
        """Parse JSON response with better error handling"""
        try:
            print("Parsing response...")
            
            # Clean up the response
            response = response.strip()
            if response.startswith('```json'):
                response = response[7:]
            if response.endswith('```'):
                response = response[:-3]
            
            # Find the JSON content
            start_idx = response.find('{')
            end_idx = response.rfind('}') + 1
            
            if start_idx == -1 or end_idx == 0:
                raise ValueError("No JSON content found in response")
                
            json_str = response[start_idx:end_idx]
            
            # Clean up the JSON string
            json_str = (
                json_str
                .replace('\n', ' ')
                .replace('\r', ' ')
                .replace('\t', ' ')
                .replace('\\', '\\\\')
                .replace('"{', '{')
                .replace('}"', '}')
                .replace("'", '"')
            )
            
            # Remove any invalid control characters
            json_str = ''.join(char for char in json_str if ord(char) >= 32 or char in '\n\r\t')
            
            # Try to parse the JSON
            try:
                result = json.loads(json_str)
            except json.JSONDecodeError:
                # If parsing fails, try to fix common issues
                json_str = json_str.replace('None', 'null')
                json_str = json_str.replace('True', 'true')
                json_str = json_str.replace('False', 'false')
                # Remove trailing commas
                json_str = json_str.replace(',}', '}')
                json_str = json_str.replace(',]', ']')
                result = json.loads(json_str)
            
            print("Successfully parsed JSON response")
            return result
                
        except Exception as e:
            print(f"Error parsing response: {str(e)}")
            print("Raw response:", response)
            # Return a default structure instead of raising an error
            if 'claims' in response.lower():
                return {"claims": []}
            elif 'evidence' in response.lower():
                return {"evidence_sets": []}
            elif 'conclusion' in response.lower():
                return {"conclusions": []}
            return {}

        

    def combine_results(self, analysis_results: Dict) -> tuple:
        """Restructure the single-pass analysis results into the desired format"""
        claims = {
            "claims": [
                {
                    "claim_id": item["claim_id"],
                    "claim_text": item["claim"]["text"],
                    "location": item["claim"]["location"],
                    "claim_type": item["claim"]["type"],
                    "exact_quote": item["claim"]["exact_quote"]
                }
                for item in analysis_results["analysis"]
            ]
        }
        
        evidence_results = [
            {
                "claim_id": item["claim_id"],
                "evidence": [
                    {
                        "evidence_id": idx + 1,
                        "evidence_text": ev["evidence_text"],
                        "evidence_type": "primary",
                        "strength": ev["strength"],
                        "limitations": ev["limitations"],
                        "location": ev["location"],
                        "exact_quote": ev["exact_quote"]
                    }
                    for idx, ev in enumerate(item["evidence"])
                ]
            }
            for item in analysis_results["analysis"]
        ]
        
        conclusions = {
            "conclusions": [
                {
                    "claim_id": item["claim_id"],
                    "author_conclusion": item["evaluation"]["justification"],
                    "conclusion_justified": item["evaluation"]["conclusion_justified"],
                    "robustness_analysis": item["evaluation"]["robustness"],
                    "limitations": item["evaluation"]["key_limitations"],
                    "evidence_alignment": item["evaluation"]["justification"],
                    "confidence_level": item["evaluation"]["confidence_level"]
                }
                for item in analysis_results["analysis"]
            ],
            "analysis_metadata": {
                "total_claims_analyzed": len(analysis_results["analysis"]),
                "claims_with_conclusions": len(analysis_results["analysis"]),
                "analysis_timestamp": str(datetime.datetime.now())
            }
        }
        
        final_results = {
            "paper_analysis": []
        }
        
        for item in analysis_results["analysis"]:
            claim_id = item["claim_id"]
            analysis = {
                "claim_id": claim_id,
                "claim": item["claim"]["text"],
                "claim_location": item["claim"]["location"],
                "evidence": item["evidence"],
                "evidence_locations": [ev["location"] for ev in item["evidence"]],
                "conclusion": {
                    "author_conclusion": item["evaluation"]["justification"],
                    "conclusion_justified": item["evaluation"]["conclusion_justified"],
                    "robustness_analysis": item["evaluation"]["robustness"],
                    "limitations": item["evaluation"]["key_limitations"],
                    "conclusion_location": item["claim"]["location"]
                }
            }
            final_results["paper_analysis"].append(analysis)
        
        return claims, evidence_results, conclusions, final_results

    def print_analysis_results(self, final_results: Dict):
        """Print the analysis results in a readable format"""
        print("\n=== Complete Paper Analysis ===\n")
        
        for analysis in final_results['paper_analysis']:
            print(f"Claim {analysis['claim_id']}:")
            print(f"Statement: {analysis['claim']}")
            print("\nEvidence:")
            for evidence in analysis['evidence']:
                print(f"- {evidence['evidence_text']}")
                print(f"  Strength: {evidence['strength']}")
                print(f"  Limitations: {evidence['limitations']}")
            
            print("\nConclusion:")
            print(f"Author's Conclusion: {analysis['conclusion']['author_conclusion']}")
            print(f"Justified by Evidence: {'Yes' if analysis['conclusion']['conclusion_justified'] else 'No'}")
            print(f"Robustness: {analysis['conclusion']['robustness_analysis']}")
            print(f"Limitations: {analysis['conclusion']['limitations']}")
            print("\n" + "-"*50 + "\n")

    def save_results(self, results: Dict, base_filename: str):
        """Save analysis results to files"""
        output_dir = Path('Gemini_all_at_once_shashi')
        output_dir.mkdir(exist_ok=True)
        

        results["execution_times"] = {
        "single_pass_analysis_time": f"{self.execution_times['single_pass_analysis']:.2f} seconds",
        "total_sleep_time": f"{self.execution_times['total_sleep_time']:.2f} seconds",
        "actual_processing_time": f"{self.execution_times['actual_processing_time']:.2f} seconds",
        "total_execution_time": f"{self.execution_times['total_time']:.2f} seconds"
        }
        # Save full JSON results
        json_path = output_dir / f'{base_filename}_analysis.json'
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=4)


        # results["execution_times"] = {
        # "single_pass_analysis_time": f"{self.execution_times['single_pass_analysis']:.2f} seconds",
        # "total_execution_time": f"{self.execution_times['total_time']:.2f} seconds"
        #  }


        
        
        # Save readable text summary
        text_path = output_dir / f'{base_filename}_summary.txt'
        with open(text_path, 'w', encoding='utf-8') as f:
            for analysis in results['analysis']:
                f.write(f"Claim {analysis['claim_id']}:\n")
                f.write(f"Type: {analysis['claim']['type']}\n")
                f.write(f"Statement: {analysis['claim']['text']}\n")
                f.write(f"Location: {analysis['claim']['location']}\n")
                f.write(f"Exact Quote: {analysis['claim']['exact_quote']}\n\n")
                
                f.write("Evidence:\n")
                for evidence in analysis['evidence']:
                    f.write(f"- Evidence Text: {evidence['evidence_text']}\n")
                    f.write(f"  Strength: {evidence['strength']}\n")
                    f.write(f"  Location: {evidence['location']}\n")
                    f.write(f"  Limitations: {evidence['limitations']}\n")
                    f.write(f"  Exact Quote: {evidence['exact_quote']}\n\n")
                
                eval_data = analysis['evaluation']
                f.write("Evaluation:\n")
                f.write(f"Conclusion Justified: {'Yes' if eval_data['conclusion_justified'] else 'No'}\n")
                f.write(f"Robustness: {eval_data['robustness']}\n")
                f.write(f"Confidence Level: {eval_data['confidence_level']}\n")
                f.write(f"Justification: {eval_data['justification']}\n")
                f.write(f"Key Limitations: {eval_data['key_limitations']}\n")
                
                f.write("\n" + "-"*50 + "\n\n")
        
        # Generate summary statistics
        stats_path = output_dir / f'{base_filename}_statistics.txt'
        with open(stats_path, 'w', encoding='utf-8') as f:
            total_claims = len(results['analysis'])
            justified_claims = sum(1 for a in results['analysis'] 
                                 if a['evaluation']['conclusion_justified'])
            
            f.write("Analysis Statistics:\n")
            f.write(f"Total Claims Analyzed: {total_claims}\n")
            f.write(f"Justified Claims: {justified_claims}\n")
            
            # Evidence strength distribution
            strength_levels = {}
            for analysis in results['analysis']:
                for evidence in analysis['evidence']:
                    strength = evidence['strength']
                    strength_levels[strength] = strength_levels.get(strength, 0) + 1
            
            f.write("\nEvidence Strength Distribution:\n")
            total_evidence = sum(strength_levels.values())
            for strength, count in strength_levels.items():
                f.write(f"{strength}: {count} pieces ({count/total_evidence*100:.1f}%)\n")


def results_exist(base_filename: str, output_folder: str) -> bool:
    """Check if results already exist for the given file."""
    output_dir = Path(output_folder)
    analysis_path = output_dir / f'{base_filename}_analysis.json'
    summary_path = output_dir / f'{base_filename}_summary.txt'
    statistics_path = output_dir / f'{base_filename}_statistics.txt'
    
    # Check if all expected output files exist
    return all(file.exists() for file in [analysis_path, summary_path, statistics_path])

def main():
    input_folder = 'shashi_1_papers'
    output_folder = 'Gemini_all_at_once_shashi'
    Path(output_folder).mkdir(exist_ok=True)  # Ensure the output directory exists

    pdf_files = [f for f in os.listdir(input_folder) if f.endswith('.pdf')]
    failed_papers = []  # To keep track of any failures

    for filename in pdf_files:
        base_filename = Path(filename).stem
        
        if results_exist(base_filename, output_folder):
            print(f"Skipping {filename}, results already exist.")
            continue
        
        filename_with_path = f"{input_folder}/{filename}"
        analyzer = SinglePassPaperAnalyzer()
        
        try:
            print(f"Starting analysis of {filename_with_path}")
            total_start_time = time.time()

            analyzer.extract_text_from_pdf(filename_with_path)
            analysis_results = analyzer.analyze_paper(filename_with_path)
            analyzer.execution_times["total_time"] = time.time() - total_start_time

            claims, evidence_results, conclusions, final_results = analyzer.combine_results(analysis_results)
            analyzer.print_analysis_results(final_results)
            analyzer.save_results(final_results, base_filename)

        except Exception as e:
            print(f"Error analyzing {filename}: {str(e)}")
            failed_papers.append(filename_with_path)

    if failed_papers:
        print("Failed to process the following papers:")
        for paper in failed_papers:
            print(paper)

if __name__ == "__main__":
    main()




Starting analysis of shashi_1_papers/2502.12568v2.pdf
Processing shashi_1_papers/2502.12568v2.pdf...
[                                        ] (0/1==[===                                     ] ( 1/1==[======                                  ] ( 2/13==[=========                               ] ( 3/1==[============                            ] ( 4/13==[===============                         ] ( 5/1==[==================                      ] ( 6/13==[=====================                   ] ( 7/1==[========================                ] ( 8/13==[===========================             ] ( 9/1==[==============================          ] (10/13==[=================================       ] (11/1==[====================================    ] (12/13===[========================================] (13/13]


ERROR:root:Error processing chunk 1: 429 Resource has been exhausted (e.g. check quota).
ERROR:root:Error processing chunk 2: 429 Resource has been exhausted (e.g. check quota).
ERROR:root:Error processing chunk 3: 429 Resource has been exhausted (e.g. check quota).
ERROR:root:Error processing chunk 4: 429 Resource has been exhausted (e.g. check quota).


Parsing response...
Error parsing response: Expecting ',' delimiter: line 1 column 822 (char 821)
Raw response: 
{
  "analysis": [
    {
      "claim_id": 1,
      "claim": {
        "text": "CogWriter demonstrates significantly faster processing compared to the baseline model.",
        "type": "performance",
        "location": "Appendix A.2, Inference Time",
        "exact_quote": "our approach demonstrates a significant reduction in generation time, achieving approximately 50% faster processing compared to the baseline model."
      },
      "evidence": [
        {
          "evidence_text": "Inference time comparison using LLaMA-3.3-70B on 4 NVIDIA A100 GPUs, with each condition tested three times and only considering outputs achieving 100% completion rate. Figure 6 illustrates the results.",
          "strength": "moderate",
          "limitations": "Limited to LLaMA-3.3-70B due to issues with other models.  Doesn't specify the exact tasks used for comparison. Only outputs with 1

ERROR:root:Error processing chunk 2: 429 Resource has been exhausted (e.g. check quota).


Parsing response...
Error parsing response: Expecting ',' delimiter: line 1 column 2011 (char 2010)
Raw response: 
{
  "analysis": [
    {
      "claim_id": 1,
      "claim": {
        "text": "Semantic equivalence across different representations holds true in our context.",
        "type": "result",
        "location": "Section 5, Hypothesis H1",
        "exact_quote": "Semantic equivalence across different representations, as discussed by Weaver, holds true in our context."
      },
      "evidence": [
        {
          "evidence_text": "Higher cosine similarity between matched pairs compared to mismatched ones using text-embedding-ada-002 and sentence-t5-xl.",
          "strength": "moderate",
          "limitations": "Relies on cosine similarity which might not fully capture semantic equivalence. Specific values not provided.",
          "location": "Section 5.2",
          "exact_quote": "both models demonstrated higher cosine similarity between matched pairs compared to mismat

...

ERROR:root:Error processing chunk 2: 429 Resource has been exhausted (e.g. check quota).
ERROR:root:Error processing chunk 3: 429 Resource has been exhausted (e.g. check quota).
ERROR:root:Error processing chunk 4: 429 Resource has been exhausted (e.g. check quota).
ERROR:root:Error processing chunk 5: 429 Resource has been exhausted (e.g. check quota).
ERROR:root:Error processing chunk 6: 429 Resource has been exhausted (e.g. check quota).


Error analyzing 2405.04215v1.pdf: No valid analyses generated from any chunk
Starting analysis of shashi_1_papers/2501.18817v1.pdf
Processing shashi_1_papers/2501.18817v1.pdf...
[                                        ] (0/2[=                                       ] ( 1/2=[===                                     ] ( 2/2[====                                    ] ( 3/25=[======                                  ] ( 4/25=[========                                ] ( 5/25[=========                               ] ( 6/2=[===========                             ] ( 7/2[============                            ] ( 8/25=[==============                          ] ( 9/25=[================                        ] (10/25[=================                       ] (11/2=[===================                     ] (12/2[====================                    ] (13/25=[======================                  ] (14/25=[========================                ] (15/25[=========================           

ERROR:root:Error processing chunk 1: 429 Resource has been exhausted (e.g. check quota).


Parsing response...
Error parsing response: Expecting ',' delimiter: line 1 column 698 (char 697)
Raw response: 
{
  "analysis": [
    {
      "claim_id": 1,
      "claim": {
        "text": "The example solution correctly solves a simple BlocksWorld task.",
        "type": "methodology",
        "location": "Section 3, Paragraph 2",
        "exact_quote": "The example solution, provided with the intent of making the solution format clear, correctly solves a simple BlocksWorld task that is not present in the experiment dataset."
      },
      "evidence": [],
      "evaluation": {
        "conclusion_justified": false,
        "robustness": "low",
        "justification": "The paper mentions the existence of an example solution and its purpose but provides no details about the task itself or how it's solved.  Without this information, the claim of its correctness cannot be verified.",
        "key_limitations": "Lack of specifics about the example solution and task.",
        "confiden

KeyboardInterrupt: 

In [40]:
import google.generativeai as genai
import json
import datetime
import pymupdf4llm
import time
from pathlib import Path
import os
import traceback
from typing import Dict, List, Any

load_dotenv()

class PaperAnalyzer:
    def __init__(self):
        api_key = os.getenv("GEMINI_AI_API_KEY")
        if not api_key:
            raise ValueError("API key not found. Please set it in the .env file.")
        

        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('gemini-1.5-pro')
        self.paper_text = None
        self.execution_times = {
        "claims_analysis": 0,
        "evidence_analysis": 0,
        "conclusions_analysis": 0,
        "total_time": 0,
        "total_sleep_time": 0,  # Track total sleep time
        "actual_processing_time": 0  # Time without sleep delays
            }
        


    def extract_text_from_pdf(self, filename: str) -> str:
        """Extract text from PDF file using PyMuPDF"""
        try:
            self.paper_text = pymupdf4llm.to_markdown(filename)
            return self.paper_text
        except Exception as e:
            print(f"Error extracting text from PDF: {e}")
            return ""
    def get_all_claims(self, filename: str) -> str:
        """Get all claims in text format"""
        try:
            start_time = time.time()

            if not self.paper_text:
                text = self.extract_text_from_pdf(filename)
            else:
                text = self.paper_text

            print(f"Processing file: {filename}")
            
            claims_prompt = f"""
            Paper text: {text}
            
            Please identify all statements that meet these criteria for a claim:
            1. Makes a specific, testable assertion about results, methods, or contributions
            2. Represents a novel finding, improvement, or advancement
            3. Presents a clear position or conclusion

            Format each claim as follows:
            Claim #: [claim text]
            Location: [section/paragraph]
            Type: [nature of claim]
            Quote: [exact quote from text]

            List all claims you can find.
            """
            
            time.sleep(90)  # Rate limiting
            self.execution_times["total_sleep_time"] += 90
            response = self.model.generate_content(claims_prompt)
            self.execution_times["claims_analysis"] = time.time() - start_time

            print("Claims extraction completed")
            return response.text
        except Exception as e:
            print(f"Error in get_all_claims: {str(e)}")
            return "Error extracting claims"

    def get_all_evidence(self, filename: str, claims_text: str) -> str:
        """Get evidence for claims in text format"""
        try:
            start_time = time.time()
            if not self.paper_text:
                text = self.extract_text_from_pdf(filename)
            else:
                text = self.paper_text
            
            evidence_prompt = f"""
            Paper text: {text}

            For these claims:
            {claims_text}

            Please identify relevant evidence for each claim that:
            1. Directly supports or contradicts the claim
            2. Is presented with experimental results or data
            3. Can be traced to specific methods or results
            4. Is not from abstract or introduction

            Format for each claim:
            Claim #: [repeat claim]
            Evidence:
            1. [evidence text]
    - Strength: [strong/moderate/weak]
    - Limitations: [key limitations]
    - Location: [section/paragraph]
    - Quote: [exact quote]

            List all evidence for each claim.
            """
            
            time.sleep(90)
            self.execution_times["total_sleep_time"] += 90
            response = self.model.generate_content(evidence_prompt)
            
            self.execution_times["evidence_analysis"] = time.time() - start_time
            print("Evidence extraction completed")
            return response.text
        except Exception as e:
            print(f"Error in get_all_evidence: {str(e)}")
            return "Error extracting evidence"

    def get_all_conclusions(self, filename: str, claims_text: str, evidence_text: str) -> str:
        """Analyze conclusions in text format"""
        try:
            start_time = time.time()
            if not self.paper_text:
                text = self.extract_text_from_pdf(filename)
            else:
                text = self.paper_text
            
            conclusions_prompt = f"""
            Paper text: {text}

            Based on these claims:
            {claims_text}

            And their evidence:
            {evidence_text}

            Please provide conclusions for each claim:
            1. Whether the evidence justifies the claim
            2. Overall strength of support
            3. Important limitations

            Format for each claim:
            Claim #: [repeat claim]
            Conclusion:
            - Justified: [yes/no]
            - Robustness: [high/medium/low]
            - Limitations: [specific limitations]
            - Confidence: [high/medium/low]
            """
            
            time.sleep(90)
            self.execution_times["total_sleep_time"] += 90
            response = self.model.generate_content(conclusions_prompt)
            
            self.execution_times["conclusions_analysis"] = time.time() - start_time
            print("Conclusions analysis completed")
            return response.text
        except Exception as e:
            print(f"Error in get_all_conclusions: {str(e)}")
            return "Error generating conclusions"

    def analyze_paper(self, filename: str) -> Dict:
        """Complete paper analysis using text-based approach"""
        try:
            total_start_time = time.time()

            print("Extracting text from PDF...")
            self.extract_text_from_pdf(filename)

            print("Extracting claims...")
            claims_text = self.get_all_claims(filename)

            print("Extracting evidence...")
            evidence_text = self.get_all_evidence(filename, claims_text)

            print("Analyzing conclusions...")
            conclusions_text = self.get_all_conclusions(filename, claims_text, evidence_text)

            # Calculate times
            total_elapsed = time.time() - total_start_time
            self.execution_times["total_time"] = total_elapsed
            self.execution_times["actual_processing_time"] = (
                total_elapsed - self.execution_times["total_sleep_time"]
            )

            return {
                "claims": claims_text,
                "evidence": evidence_text,
                "conclusions": conclusions_text,
                "execution_times": self.execution_times
            }

        except Exception as e:
            print(f"Error in paper analysis: {str(e)}")
            return None

    def save_results(self, results: Dict, filename: str):
        """Save analysis results in text format"""
        try:
            base_filename = filename
            output_dir = "Gemini_3_prompts_shashi"
            os.makedirs(output_dir, exist_ok=True)

            # Save all results in a single text file
            output_filename = f'{output_dir}/{base_filename}_analysis.txt'
            with open(output_filename, 'w', encoding='utf-8') as f:
                f.write("=== PAPER ANALYSIS RESULTS ===\n\n")
                
                f.write("=== CLAIMS ===\n")
                f.write(results['claims'])
                f.write("\n\n")
                
                f.write("=== EVIDENCE ===\n")
                f.write(results['evidence'])
                f.write("\n\n")
                
                f.write("=== CONCLUSIONS ===\n")
                f.write(results['conclusions'])
                f.write("\n\n")
                
                f.write("=== EXECUTION TIMES ===\n")
                f.write(f"Claims Analysis Time: {results['execution_times']['claims_analysis']:.2f} seconds\n")
                f.write(f"Evidence Analysis Time: {results['execution_times']['evidence_analysis']:.2f} seconds\n")
                f.write(f"Conclusions Analysis Time: {results['execution_times']['conclusions_analysis']:.2f} seconds\n")
                f.write(f"Total Sleep Time: {results['execution_times']['total_sleep_time']:.2f} seconds\n")
                f.write(f"Actual Processing Time: {results['execution_times']['actual_processing_time']:.2f} seconds\n")
                f.write(f"Total Time: {results['execution_times']['total_time']:.2f} seconds\n")

            print(f"Results saved to: {output_filename}")

        except Exception as e:
            print(f"Error saving results: {str(e)}")


def main():
    analyzer = PaperAnalyzer()
    
    # filename = "Ax_Hao_Hang_2.pdf"

    input_folder = 'shashi_1_papers'

    pdf_files = [f for f in os.listdir(input_folder) if f.endswith('.pdf')]

    for filename in pdf_files:
        basefile_name = Path(filename).stem
        try:
            filename = f"{input_folder}/{filename}"
            print(f"Starting analysis of {filename}")
                
            # Analyze paper
            results = analyzer.analyze_paper(filename)
            
            if results:
                # Save results in structured format
                analyzer.save_results(results, basefile_name)
                print("Analysis completed successfully")
            else:
                print("Analysis failed to produce results")
                
        except Exception as e:
            print(f"Error in main execution: {str(e)}")
            traceback.print_exc()

        
    #     print(f"Starting analysis of {filename}")
        
    #     # Analyze paper
    #     results = analyzer.analyze_paper(filename)
        
    #     if results:
    #         # Save results in structured format
    #         analyzer.save_results(results, filename)
    #         print("Analysis completed successfully")
    #     else:
    #         print("Analysis failed to produce results")
            
    # except Exception as e:
    #     print(f"Error in main execution: {str(e)}")
    #     traceback.print_exc()

if __name__ == "__main__":
    main()



Starting analysis of shashi_1_papers/2502.12568v2.pdf
Extracting text from PDF...
Processing shashi_1_papers/2502.12568v2.pdf...
[                                        ] (0/1==[===                                     ] ( 1/1==[======                                  ] ( 2/13==[=========                               ] ( 3/1==[============                            ] ( 4/13==[===============                         ] ( 5/1==[==================                      ] ( 6/13==[=====================                   ] ( 7/1==[========================                ] ( 8/13==[===========================             ] ( 9/1==[==============================          ] (10/13==[=================================       ] (11/1==[====================================    ] (12/13===[========================================] (13/13]
Extracting claims...
Processing file: shashi_1_papers/2502.12568v2.pdf
Error in get_all_claims: 429 Resource has been exhausted (e.g. check quota).
Extracting evide